In [2]:
import pandas as pd
import tweepy
import csv

In [12]:
consumer_key = "Y6e6vo4pw6sX2o3PtZhJQPNxB" 
consumer_secret = "haYmIktsZbpu9fswSwKL7kxGQqstAi1vatGk9kMcIlBrdF7kxT" 
access_token = "1324898678767824897-f8LQps1yYauqKDdfVRF1CPLSMXSJQn" 
access_token_secret = "ao6Hgp8kQnk4Xfbw8szeZfCMJM11EKUSx2RNBaCpO39of"

In [13]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [14]:
#Typed following into terminal to create the text file:
#snscrape twitter-search "#tesla since:2021-02-01 until:2021-02-09" > tesla_feb21.txt
#this looks up all tweets that include "#tesla" the week that his 2016 album came out

In [15]:
tweet_url = pd.read_csv("tesla_feb21.txt", index_col= None,#make sure correct*************
header = None, names = ["links"])

af = lambda x: x["links"].split("/")[-1]
tweet_url['id'] = tweet_url.apply(af, axis=1)
tweet_url.head()

,links,id
0,https://twitter.com/SomosNDG/status/1358928501...,1358928501848612865
1,https://twitter.com/terryfrost/status/13589282...,1358928294247239682
2,https://twitter.com/kashira_s/status/135892828...,1358928285359476743
3,https://twitter.com/justindanethan/status/1358...,1358928283501531141
4,https://twitter.com/viaggrego/status/135892828...,1358928283056943106


In [16]:
ids = tweet_url['id'].tolist()
total_count = len(ids)
chunks = (total_count - 1) // 50 + 1

def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"date": status.created_at,
                     #"tweet_id":status.id,
                     "tweet":status.full_text,
                     #"User location":status.user.location,
                     "Retweet count":status.retweet_count,
                     "Like count":status.favorite_count,
                     #"Source":status.source
                    }
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    empty_data.to_csv("tesla_jan21.csv", mode="a") 

for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        result = fetch_tw(batch)

In [17]:
import os
import glob
import pandas as pd
os.chdir("/Users/griffinclark/Documents/Clubs/QMIND/TweetStock/2021")

In [18]:
#if there are multiple csv files that you want combined:
#go to directory where all your csv files you want to combine are
#use following code in terminal:
# cat *.csv >teslaNew.csv

df = pd.read_csv("/Users/griffinclark/Documents/Clubs/QMIND/TweetStock/2021/teslaNew.csv", engine = 'python')

In [19]:
#df.drop('Unnamed: 0', inplace=True, axis=1)
df.head(10)

,Unnamed: 0,Like count,Retweet count,date,tweet,Clean
0,0,1.0,1.0,2021-02-08 13:26:15,See that’s is what impact of a institutional i...,see that is what impact of institutional inves...
1,1,1.0,0.0,2021-02-08 13:26:14,"Tesla bought $1.5 billion in Bitcoin, expects ...",tesla bought billion in bitcoin expects to beg...
2,2,0.0,0.0,2021-02-08 13:26:29,$btc #Tesla https://t.co/CaUmpIeFul,btc tesla
3,3,1.0,0.0,2021-02-08 13:26:38,Ready for another #Bitcoin rally?\n\n#Tesla #E...,ready for another bitcoin rally tesla elonmusk...
4,4,0.0,0.0,2021-02-08 13:26:58,#bitcoin welcomes #Tesla when $FB $AAPL $AMZN ...,bitcoin welcomes tesla when fb aapl amzn googl
5,5,11.0,7.0,2021-02-08 13:26:39,"#Tesla Inc invests $1.5 billion in #bitcoin, t...",tesla inc invests billion in bitcoin the curre...
6,6,0.0,0.0,2021-02-08 13:26:45,This is a sad day for $TSLA investors :( \n#BT...,this is sad day for tsla investors btc is not ...
7,7,0.0,0.0,2021-02-08 13:26:41,@theprernaa Buy bitcoin and see magic #Tesla,buy bitcoin and see magic tesla
8,8,0.0,0.0,2021-02-08 13:26:32,Me trying to log in to Binance to buy more BTC...,me trying to log in to binance to buy more btc...
9,9,0.0,0.0,2021-02-08 13:26:28,@Tesla just announced #bitcoin will be accepta...,just announced bitcoin will be acceptable as p...


In [13]:
import re
import urllib3
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import time
from itertools import islice
import analyzer

In [14]:
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9_]+' # For removing mentions.
pat2 = r'https?://[^ ]+' # For removing links.
combined_pat = r'|'.join((pat1, pat2)) # For removing links.
www_pat = r'www.[^ ]+' # For removing links.
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"} # For removing negations.
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b') # For removing negations.
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
def tweet_cleaner(text):
      """ This cleans the tweets text content
          so it can be machine readable.
      """
      soup = BeautifulSoup(text, 'lxml') # Decodes HTML.
      souped = soup.get_text()
      try: # UTF decoding.
          bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
      except:
          bom_removed = souped
      stripped = re.sub(combined_pat, '', bom_removed) # Gets rid of links.
      stripped = re.sub(www_pat, '', stripped) # Gets rid of links.
      lower_case = stripped.lower()
      neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
      letters_only = re.sub("[^a-zA-Z]", " ", neg_handled) # Only lets letters in.
      words = [x for x  in tok.tokenize(letters_only) if len(x) > 1] # Below removes whitespace
      return (" ".join(words)).strip()

In [15]:
#drop any columns you deem unnecessary
df.drop('Source', inplace=True, axis=1)
df.drop('Unnamed: 0', inplace=True, axis=1)
df.drop('tweet_id', inplace=True, axis=1)
df.drop('User location', inplace = True, axis = 1) #delete columns we don't need
df.head()

KeyError: "['Source'] not found in axis"

In [16]:
tic = time.perf_counter()

data = []
for index, row in df.iterrows():
    data.append(tweet_cleaner(str(row["tweet"])))
df["Clean"] = data
df.to_csv("TeslaNew.csv")

toc = time.perf_counter()
print(f"Cleaned {len(df)} in {toc - tic:0.4f} seconds")
df.head()

Cleaned 58806 in 19.9378 seconds


,Like count,Retweet count,date,tweet,Clean
0,1.0,1.0,2021-02-08 13:26:15,See that’s is what impact of a institutional i...,see that is what impact of institutional inves...
1,1.0,0.0,2021-02-08 13:26:14,"Tesla bought $1.5 billion in Bitcoin, expects ...",tesla bought billion in bitcoin expects to beg...
2,0.0,0.0,2021-02-08 13:26:29,$btc #Tesla https://t.co/CaUmpIeFul,btc tesla
3,1.0,0.0,2021-02-08 13:26:38,Ready for another #Bitcoin rally?\n\n#Tesla #E...,ready for another bitcoin rally tesla elonmusk...
4,0.0,0.0,2021-02-08 13:26:58,#bitcoin welcomes #Tesla when $FB $AAPL $AMZN ...,bitcoin welcomes tesla when fb aapl amzn googl


In [18]:
len(df)

58806

In [19]:
tic = time.perf_counter()
j = len(df)
length = 0

try:
    for index, row in islice(df.iterrows(), index, None):
        length = row
        if detect(row["Clean"])!= 'en': #this should work?
            df.drop(index, inplace=True)
except:
    df.drop(index, inplace=True)
    print("row", row, "is not able to be determined")

row Like count                                                    21.0
Retweet count                                                  0.0
date                                           2021-02-08 13:27:46
tweet            #Tesla 1.5 milyar$ lık #Bitcoin almış, sanırım...
Clean            tesla milyar bitcoin alm san bir re sonra bitc...
Name: 58805, dtype: object is not able to be determined


In [20]:
df.to_csv("Tesla_Final_Data.csv")
df = pd.read_csv("Tesla_Final_Data.csv")

In [21]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import twitter_samples 
 
nltk.download('twitter_samples')
nltk.download('vader_lexicon')
 
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/griffinclark/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/griffinclark/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
i = 0
neg = []
neu = []
pos = []
analysis = []

for i in range(len(df)-1):
    try:
        analysis.append(analyzer.polarity_scores(df["Clean"][i]))
    except:
        analysis.append({'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0})
        i+=1

len(analysis)

58804

In [41]:
df['neg'] = None
df['neu'] = None
df['pos'] = None
df['compound'] = None

In [42]:
i = 0
for i in range(len(analysis)-1):
    df['neg'][i] = analysis[i]['neg']
    df['neu'][i] = analysis[i]['neu']
    df['pos'][i] = analysis[i]['pos']
    df['compound'][i] = analysis[i]['compound']

In [44]:
df.head()
df.to_csv("Tesla_Final_Data.csv")